In [1]:
import os
from rdkit import Chem
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from utils import get_experiments

In [2]:
d = "slga_hyper3"
e1 = get_experiments(d)
d = "slga_hyper1"
e2 = get_experiments(d)
e2 = e2[(e2.Pc == 0.1)]
e = pd.concat([e1, e2])
e.Nits = e.Nits.astype(int)
palette = sns.color_palette("Spectral", 6)[::5]

In [ ]:
sns.set_theme()
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for k, (t, subset) in enumerate(e.groupby("Target")):
    # Generation docking scores
    combined = None
    ax = axes[k,0]
    for _, row in subset.iterrows():
        df = pd.concat([pd.read_parquet(f"{row.path}/gen_{n}.parquet") for n in range(1,row.Nits+1)])
        df["Start"] = row.Start
        combined = pd.concat([combined, df])
    sns.lineplot(combined, x="generation", y="scores", hue="Start", palette=palette, ax=ax)
    ax.set_ylabel("-(Docking Scores)")
    ax.set_xlabel("Generation")
    ax.set_xlim(0,50)
    ax.set_title(f"{t} - Docking scores of each generation")
    ax = axes[k,1]

    # Population docking scores
    combined = None
    for _, row in subset.iterrows():
        df = pd.concat([pd.read_parquet(f"{row.path}/pop_{n}.parquet") for n in range(1,row.Nits+1)])
        df["Start"] = row.Start
        combined = pd.concat([combined, df])
    sns.lineplot(combined, x="generation", y="scores", hue="Start", palette=palette, ax=ax)
    ax.set_ylabel("-(Docking Scores)")
    ax.set_xlabel("Generation")
    ax.set_xlim(0,50)
    ax.set_title(f"{t} - Docking scores of each population")

    # Violin plot final population
    ax = axes[k,2]
    combined = None
    for _, row in subset.iterrows():
        df = pd.read_parquet(f"{row.path}/pop_{row.Nits}.parquet")
        df["Start"] = row.Start
        combined = pd.concat([combined, df])
    sns.violinplot(combined, y="scores", x="Start", inner="quart", ax=ax, color=palette[0])
    ax.set_title(f"{t} - Distribution of docking scores in final population")
    ax.set_ylabel("-(Docking Scores)")

plt.tight_layout()
plt.savefig("../figures/slga_start.pdf")